In [1]:
DATA_DIR = '/media/shreyas/DATA/ML_DATA/dogsvscats/'
#DATA_DIR = '/media/shreyas/DATA/ML_DATA/dogsvscats/sample/'
traindata_path = DATA_DIR + 'train/'
validdata_path = DATA_DIR + 'valid/'
testdata_path = '/media/shreyas/DATA/ML_DATA/dogsvscats/test/'
results_path = '/media/shreyas/DATA/ML_DATA/dogsvscats/results/'

In [2]:
from __future__ import division, print_function

import os, json
from glob import glob
import numpy as np
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.applications import VGG16

%matplotlib inline
from utils import *
from vgg16 import Vgg16


Using TensorFlow backend.


### Generate Batches

In [3]:
batch_size = 32
epochs = 3

In [4]:
def get_batches(path, class_mode='categorical', gen=image.ImageDataGenerator(), \
                shuffle=True, target_size=(224,224), batch_size=1):
    return gen.flow_from_directory(path, class_mode=class_mode, batch_size=batch_size, \
                                   target_size=target_size, shuffle=shuffle)

def get_steps(batches, batch_size):
    steps = int(batches.samples/batch_size)
    return (steps if batches.samples%batch_size==0 else (steps+1))


In [5]:
train_b = get_batches(traindata_path, batch_size=batch_size)
valid_b = get_batches(validdata_path, batch_size=batch_size)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [6]:
train_labels = train_b.classes
valid_labels = valid_b.classes

In [7]:
y_train = keras.utils.to_categorical(train_labels)
y_valid = keras.utils.to_categorical(valid_labels)

In [8]:
print(train_labels.shape, valid_labels.shape, y_train.shape, y_valid.shape)

(23000,) (2000,) (23000, 2) (2000, 2)


### 1. Create VGG16 with custom Top Layer

In [9]:
def vgg_conv():
    conv_model = VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))
    return conv_model


def top_layer(input_shape):
    model = Sequential()
    model.add(Flatten(input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dense(2, activation='softmax'))
    
    return model 

In [10]:
trn_b = get_batches(traindata_path, class_mode=None, shuffle=False, batch_size=batch_size)
val_b = get_batches(validdata_path, class_mode=None, shuffle=False, batch_size=batch_size)
tst_b = get_batches(testdata_path, class_mode=None, shuffle=False, batch_size=batch_size)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [11]:
trn_steps = get_steps(train_b, batch_size=batch_size)
val_steps = get_steps(valid_b, batch_size=batch_size)
tst_steps = get_steps(tst_b, batch_size=batch_size)

### 1.1 Extract features

In [ ]:
vgg_conv = vgg_conv()
vgg_conv.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

trn_ft = vgg_conv.predict_generator(trn_b, trn_steps)
val_ft = vgg_conv.predict_generator(val_b, val_steps)
#tst_ft = vgg_conv.predict_generator(tst_b, tst_steps)

In [ ]:
print(trn_ft.shape, val_ft.shape)

In [ ]:
save_array(results_path + 'trn_ft.dat', trn_ft)
save_array(results_path + 'val_ft.dat', val_ft)

### 1.2 Train Top Layer

In [ ]:
X_train = load_array(results_path + 'trn_ft.dat')
X_valid = load_array(results_path + 'val_ft.dat')

In [ ]:
model = top_layer(X_train.shape[1:])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=50, batch_size=128, verbose=1, validation_data=(X_valid, y_valid))

In [ ]:
model.save_weights(results_path+'bottleneck.h5')

### 1.3 Fine tune the Model

In [12]:
base_model = vgg_conv()
for layer in base_model.layers[:15]: layer.trainable=False


In [13]:
top_model = top_layer(base_model.output_shape[1:])

In [14]:
top_model.load_weights(results_path+'bottleneck.h5')

In [15]:
output = top_model(base_model.output)

In [16]:
vgg_model = Model(inputs=base_model.input, outputs=output)

In [17]:
vgg_model.load_weights(results_path+'vgg_notop.h5')

In [18]:
vgg_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
vgg_model.evaluate_generator(valid_b, val_steps)

[0.060157823826186362, 0.97799999999999998]

In [ ]:
filepath = results_path+'vgg_notop.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True,\
                             save_weights_only=True, mode='min', period=1)
callbacks=[checkpoint]

In [ ]:
vgg_model.fit_generator(train_b, steps_per_epoch=trn_steps, epochs=20, callbacks=callbacks,\
                        validation_data=valid_b, validation_steps=val_steps)

### 2. VGG16 Plus 2 Dense layers

In [22]:
model = VGG16(weights='imagenet')

In [23]:
model.layers.pop()
for layer in model.layers: layer.trainable=False
x = model.output
ouput_layer = Dense(2, activation='softmax')(x)
vgg = Model(inputs=model.input, outputs=ouput_layer)

In [24]:
vgg.load_weights(results_path+'vgg_plus2.h5')

In [25]:
vgg.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
vgg.evaluate_generator(valid_b, val_steps)

[0.11020000743865967, 0.97150000000000003]

In [ ]:
filepath = results_path+'vgg_plus2.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True,\
                             save_weights_only=True, mode='min', period=1)
callbacks=[checkpoint]

In [ ]:
vgg.fit_generator(train_b, steps_per_epoch=trn_steps, epochs=20, callbacks=callbacks,\
                        validation_data=valid_b, validation_steps=val_steps)

### 3. VGG16 with custom output layer

In [27]:
model = VGG16(weights='imagenet')
# get the index for last conv layer
layers = model.layers
last_dense_idx = [index for index,layer in enumerate(layers) if type(layer) is Dense][-1]

In [28]:
layers = layers[:last_dense_idx]
for layer in layers: layer.trainable = False


In [29]:
vgg2 = Sequential(layers)

In [30]:
vgg2.add(Dense(2, activation='softmax'))

In [31]:
vgg2.load_weights(results_path+'vgg2.h5')

In [32]:
vgg2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [33]:
vgg2.evaluate_generator(valid_b, val_steps)

[0.15025492376799229, 0.96199999999999997]

In [ ]:
filepath = results_path+'vgg2.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True,\
                             save_weights_only=True, mode='min', period=1)
callbacks=[checkpoint]

In [ ]:
vgg2.fit_generator(train_b, steps_per_epoch=trn_steps, epochs=20, callbacks=callbacks,\
                        validation_data=valid_b, validation_steps=val_steps)

### 3.3 VGG fine tune toplayer

In [34]:
model = VGG16(weights='imagenet')
# get the index for last conv layer
layers = model.layers
last_dense_idx = [index for index,layer in enumerate(layers) if type(layer) is Dense][-1]

In [35]:
layers = layers[:last_dense_idx]
last_conv_idx = [index for index,layer in enumerate(layers) if type(layer) is Conv2D][-1]
for layer in layers[:last_conv_idx]: layer.trainable = False


In [36]:
vgg_conv = Sequential(layers)

In [37]:
vgg_conv.add(Dense(2, activation='softmax'))

In [38]:
vgg_conv.load_weights(results_path+'vgg_conv.h5')

In [39]:
vgg_conv.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [40]:
vgg_conv.evaluate_generator(valid_b, val_steps)

[7.8333944778442381, 0.51400000000000001]

In [ ]:
filepath = results_path+'vgg_conv.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True,\
                             save_weights_only=True, mode='min', period=1)
callbacks=[checkpoint]

In [ ]:
vgg_conv.fit_generator(train_b, steps_per_epoch=trn_steps, epochs=20, callbacks=callbacks,\
                        validation_data=valid_b, validation_steps=val_steps)

In [ ]:
print (preds[:5])
img = batches.filenames
print (img[:5])

In [ ]:
from PIL import Image
Image.open(DATA_DIR+'test/'+img[2])

In [ ]:
save_array(results_path + 'test_predictions.dat', preds)
save_array(results_path + 'imagefiles.dat', img)

### Validate Predictions
Lets plot -
1. A few correct labels at random
2. A few incorrect labels at random
3. Most confident correct predictions of each class
4. Most confident incorrect predictions of each class
5. Most uncertain labels (probabilites close to 0.5)

In [ ]:
vgg.model.load_weights(filepath)

In [ ]:
val_batches, probs = vgg.test(validdata_path, batch_size=batch_size*2)

In [ ]:
img = val_batches.filenames
expected_labels = val_batches.classes

our_predictions = probs[:,0]
other_predictions = np.round(probs[:,1])
our_labels = np.round(1-our_predictions)

In [ ]:
from keras.preprocessing import image

def plots_idx(idx, titles=None):
    plots([image.load_img(validdata_path + img[i]) for i in idx], titles=titles)

In [ ]:
n_view = 4

In [ ]:
correct = np.where(our_labels==expected_labels)[0]
print ("Found %d correct labels" % len(correct))
idx = permutation(correct)[:n_view]
plots_idx(idx, our_predictions[idx])

In [ ]:
incorrect = np.where(our_labels!=expected_labels)[0]
print ("Found %d incorrect labels" % len(incorrect))
idx = permutation(incorrect)[:n_view]
plots_idx(idx, our_predictions[idx])

In [ ]:

#3a. The images we most confident were cats, and are actually cats
correct_cats = np.where((our_labels==0) & (our_labels==expected_labels))[0]
print ("Found %d confident correct cats labels" % len(correct_cats))
most_correct_cats = np.argsort(our_predictions[correct_cats])[::-1][:n_view]
plots_idx(correct_cats[most_correct_cats], our_predictions[correct_cats][most_correct_cats])

In [ ]:
#3b. The images we most confident were dogs, and are actually dogs
correct_dogs = np.where((our_labels==1) & (our_labels==expected_labels))[0]
print ("Found %d confident correct dogs labels" % len(correct_dogs))
most_correct_dogs = np.argsort(our_predictions[correct_dogs])[:n_view]
plots_idx(correct_dogs[most_correct_dogs], our_predictions[correct_dogs][most_correct_dogs])

In [ ]:
#4a. The images we were most confident were cats, but are actually dogs
incorrect_cats = np.where((our_labels==0) & (our_labels!=expected_labels))[0]
print ("Found %d incorrect cats" % len(incorrect_cats))
if len(incorrect_cats):
    most_incorrect_cats = np.argsort(our_predictions[incorrect_cats])[::-1][:n_view]
    plots_idx(incorrect_cats[most_incorrect_cats], our_predictions[incorrect_cats][most_incorrect_cats])

In [ ]:
#4b. The images we were most confident were dogs, but are actually cats
incorrect_dogs = np.where((our_labels==1) & (our_labels!=expected_labels))[0]
print ("Found %d incorrect dogs" % len(incorrect_dogs))
if len(incorrect_dogs):
    most_incorrect_dogs = np.argsort(our_predictions[incorrect_dogs])[:n_view]
    plots_idx(incorrect_dogs[most_incorrect_dogs], our_predictions[incorrect_dogs][most_incorrect_dogs])

In [ ]:
#5. The most uncertain labels (ie those with probability closest to 0.5).
most_uncertain = np.argsort(np.abs(our_predictions-0.5))
plots_idx(most_uncertain[:n_view], our_predictions[most_uncertain])

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(expected_labels, our_labels)
plot_confusion_matrix(cm, val_batches.class_indices)

In [ ]:
preds = load_array(results_path + 'test_predictions.dat')
filenames = load_array(results_path + 'imagefiles.dat')

In [ ]:
isdog = preds[:,1]
print ("Raw Predictions: "+ str(isdog[:5]))
print ("Mid Predictions: "+str(isdog[(isdog<.6)&(isdog>0.4)]))
print ("Edge Predictions: "+str(isdog[(isdog<0.02)&(isdog>.98)]))

In [ ]:
np.amax(isdog)

In [ ]:
#Extract imageIds from the filenames in our test/unknown directory 
filenames = batches.filenames
ids = np.array([int(f[8:f.find('.')]) for f in filenames])

In [ ]:
subm = np.stack([ids,isdog], axis=1)
subm[:5]

In [ ]:
%cd $DATA_DIR
submission_file_name = 'submission1.csv'
np.savetxt(submission_file_name, subm, fmt='%d,%.5f', header='id,label', comments='')